In [2]:
%%shell
# Clone ThunderSVM repository
git clone https://github.com/Xtra-Computing/thundersvm.git

# Build ThunderSVM
cd thundersvm
mkdir build
cd build
cmake ..
make -j

Cloning into 'thundersvm'...
remote: Enumerating objects: 7469, done.
remote: Counting objects: 100% (478/478), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 7469 (delta 266), reused 440 (delta 248), pack-reused 6991 (from 1)
Receiving objects: 100% (7469/7469), 4.88 MiB | 7.80 MiB/s, done.
Resolving deltas: 100% (4996/4996), done.
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- D

In [1]:
from importlib.machinery import SourceFileLoader
thundersvm = SourceFileLoader("thundersvm", "/content/thundersvm/python/thundersvm/thundersvm.py").load_module()


In [2]:
!pip install scikit-optimize

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import StandardScaler as stdscale
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
import csv
from skopt import BayesSearchCV


In [4]:

file_name = '/content/2020_stat.csv'
table = pd.read_csv(file_name, quoting=csv.QUOTE_NONE, on_bad_lines='skip').dropna()


In [5]:
for col in table.columns:
  table.columns = table.columns.str.replace('"', '')
table.columns

Index(['LOAN_ID', 'ORIG_RATE', 'ORIG_AMOUNT', 'ORIG_TERM', 'OLTV', 'NUM_BO',
       'DTI', 'CSCORE_B', 'CSCORE_C', 'FTHB_FLAG', 'PUR_Cash_out',
       'PUR_Refinance', 'PUR_Purchase', 'PRO_Condominium', 'PRO_Co_operative',
       'PRO_Planned_Urban', 'PRO_Manufact_Home', 'PRO_Single_Family',
       'NUM_UNIT', 'OCC_Principal', 'OCC_Second', 'OCC_Investor', 'STATE',
       'ZIP', 'MSA', 'MI_PCT', 'MI_Borrower', 'MI_Lender', 'MI_Investor',
       'AQSN_DTE', 'ORIG_DTE', 'FRST_DTE', 'LAST_RATE', 'LAST_UPB',
       'LAST_ACTIVITY_DATE', 'DLQ_30_FLAG', 'DLQ_60_FLAG', 'DLQ_90_FLAG',
       'DLQ_120_FLAG', 'Ongoing', 'Current_DLQ', 'Prepaid_Matured',
       'Third_Party_Sale', 'Short_Sale', 'Repurchased', 'Deed_In_Lieu',
       'Non_Performing_NS', 'Reperforming_NS', 'COMPLETE_FLAG', 'NET_LOSS',
       'NET_SEV'],
      dtype='object')

In [6]:
model_data = table[[ 'ORIG_RATE', 'ORIG_AMOUNT', 'ORIG_TERM', 'OLTV', 'NUM_BO', 'DTI', 'CSCORE_B','FTHB_FLAG', 'PUR_Cash_out', 'PUR_Refinance', 'PUR_Purchase', 'NUM_UNIT', 'OCC_Principal', 'OCC_Second', 'OCC_Investor', 'MI_PCT', 'DLQ_30_FLAG', 'DLQ_90_FLAG', 'Ongoing', 'Current_DLQ', 'Prepaid_Matured']]
for col in model_data.select_dtypes(include=['object']).columns:
    model_data[col] = model_data[col].str.replace('"', '', regex=False)
    model_data[col] = pd.to_numeric(model_data[col], errors='coerce')
model_data = model_data.dropna()
model_data

<ipython-input-6-0c4c40ba9257>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data[col] = model_data[col].str.replace('"', '', regex=False)
<ipython-input-6-0c4c40ba9257>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data[col] = pd.to_numeric(model_data[col], errors='coerce')


,ORIG_RATE,ORIG_AMOUNT,ORIG_TERM,OLTV,NUM_BO,DTI,CSCORE_B,FTHB_FLAG,PUR_Cash_out,PUR_Refinance,...,NUM_UNIT,OCC_Principal,OCC_Second,OCC_Investor,MI_PCT,DLQ_30_FLAG,DLQ_90_FLAG,Ongoing,Current_DLQ,Prepaid_Matured
0,3.38,308000.0,180,75.0,1,38.0,796.0,1,0,0,...,1,1,0,0,0.0,0,0,0,0,1
1,3.62,207000.0,240,66.0,2,25.0,806.0,0,0,1,...,1,1,0,0,0.0,0,0,0,0,1
2,3.62,215000.0,240,73.0,2,42.0,745.0,0,0,1,...,1,1,0,0,0.0,0,0,0,0,1
3,4.00,319000.0,360,75.0,2,25.0,774.0,0,0,1,...,1,1,0,0,0.0,0,0,0,0,1
4,4.12,362000.0,360,78.0,1,38.0,763.0,0,0,1,...,1,1,0,0,0.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4817642,2.88,136000.0,360,46.0,1,45.0,804.0,0,0,1,...,1,1,0,0,0.0,0,0,1,0,0
4817643,2.99,208000.0,360,64.0,2,38.0,751.0,0,1,0,...,1,1,0,0,0.0,0,0,0,0,1
4817644,3.12,214000.0,360,94.0,1,34.0,712.0,0,0,1,...,1,1,0,0,30.0,1,0,0,0,1
4817645,3.38,118000.0,360,80.0,1,43.0,660.0,1,0,0,...,1,1,0,0,0.0,0,0,1,0,0


In [7]:

# Definition of Bad Loans: once had a 90-day delinquency in performance history
# Definition of Good Loans: no delinquency and continuous payments up to current
model_data = model_data[ (model_data['DLQ_90_FLAG'] == 1) | ( (model_data['DLQ_90_FLAG'] == 0) & (model_data['Ongoing'] == 1) ) ]
model_data = model_data.astype(float)
X = model_data.drop(columns=['DLQ_30_FLAG', 'DLQ_90_FLAG', 'Ongoing', 'Current_DLQ', 'Prepaid_Matured'])
y = model_data['DLQ_90_FLAG']

In [8]:

num_col = ['ORIG_RATE', 'ORIG_AMOUNT', 'ORIG_TERM', 'OLTV', 'NUM_BO', 'DTI', 'CSCORE_B', # 'CSCORE_C',
           'NUM_UNIT', 'MI_PCT']
cat_col = ['FTHB_FLAG', 'PUR_Cash_out', 'PUR_Refinance', 'PUR_Purchase', 'OCC_Principal', 'OCC_Second', 'OCC_Investor']
scaler = stdscale()
X[num_col] = scaler.fit_transform(X[num_col])
X = pd.concat([X[num_col], X[cat_col]], axis=1)

X

,ORIG_RATE,ORIG_AMOUNT,ORIG_TERM,OLTV,NUM_BO,DTI,CSCORE_B,NUM_UNIT,MI_PCT,FTHB_FLAG,PUR_Cash_out,PUR_Refinance,PUR_Purchase,OCC_Principal,OCC_Second,OCC_Investor
5,5.967575,-0.041969,0.583236,0.558688,-0.961138,1.209533,-2.728869,-0.110778,-0.514104,0.0,1.0,0.0,0.0,1.0,0.0,0.0
10,2.077307,0.221726,0.583236,-0.005569,-0.961138,-0.404656,-2.285486,-0.110778,-0.514104,0.0,1.0,0.0,0.0,1.0,0.0,0.0
14,1.311669,1.306641,0.583236,0.558688,0.975992,-0.101995,-0.709015,-0.110778,-0.514104,0.0,0.0,1.0,0.0,1.0,0.0,0.0
18,1.580677,-1.013873,0.583236,1.405074,-0.961138,-0.909090,0.596500,-0.110778,2.219114,1.0,0.0,0.0,1.0,1.0,0.0,0.0
19,1.808299,-1.435785,0.583236,1.122946,0.975992,-2.422392,0.744294,-0.110778,1.763577,1.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4817638,-2.557905,-0.689905,-1.827631,-2.939707,-0.961138,-0.909090,0.399441,-0.110778,-0.514104,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4817639,-1.274945,0.372408,0.583236,-0.795529,-0.961138,-1.817071,0.547235,-0.110778,-0.514104,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4817642,-0.488614,-1.021407,0.583236,-1.359786,-0.961138,1.209533,0.941353,-0.110778,-0.514104,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4817645,0.546031,-1.157022,0.583236,0.558688,-0.961138,1.007759,-2.605707,-0.110778,-0.514104,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)


In [12]:
len(X_train)

2282065

In [13]:
clf = thundersvm.SVC(kernel='rbf', C=1.0, class_weight='balanced', verbose=True)
clf.fit(X_train[:500000], y_train[:500000])



SVC(class_weight='balanced', verbose=True)

In [14]:
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"AUC-ROC: {round(roc_auc, 4)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.7513
AUC-ROC: 0.7641
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.75      0.85    950161
         1.0       0.08      0.78      0.15     27867

    accuracy                           0.75    978028
   macro avg       0.54      0.76      0.50    978028
weighted avg       0.97      0.75      0.83    978028



In [22]:
joblib.dump(clf, '500k_samples.joblib')


['500k_samples.joblib']

In [10]:
param_space = {
    'C': (1e-2, 1e+2, 'log-uniform'),
    'gamma': (1e-3, 1e-1, 'log-uniform'),
    'kernel': ['linear']
}

model = thundersvm.SVC(class_weight='balanced')

# bayesian optimization search
opt = BayesSearchCV(model, param_space, n_iter=20, cv=3, random_state=42, verbose=4)
opt.fit(X_train[:10000], y_train[:10000])

print("Best parameters:", opt.best_params_)
print("Best cross-validation score:", opt.best_score_)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END C=0.4369339947510315, gamma=0.02853983686604182, kernel=linear;, score=0.754 total time=   0.9s
[CV 2/3] END C=0.4369339947510315, gamma=0.02853983686604182, kernel=linear;, score=0.761 total time=   0.5s
[CV 3/3] END C=0.4369339947510315, gamma=0.02853983686604182, kernel=linear;, score=0.745 total time=   0.5s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END C=22.364202820542705, gamma=0.058429282697611454, kernel=linear;, score=0.754 total time=  15.1s
[CV 2/3] END C=22.364202820542705, gamma=0.058429282697611454, kernel=linear;, score=0.760 total time=  14.5s
[CV 3/3] END C=22.364202820542705, gamma=0.058429282697611454, kernel=linear;, score=0.745 total time=  15.5s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END C=0.6016307829589929, gamma=0.06877728743793542, kernel=linear;, score=0.753 total time=   0.6s
[CV 2/3] END C=0.6016307829589929, gamma=0.06877728743793

In [11]:
# Best parameters: OrderedDict([('C', 0.011102560821691395), ('gamma', 0.001), ('kernel', 'linear')]
svm_model = LinearSVC(random_state=99, max_iter=8192, C=0.011102560821691395, class_weight='balanced')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"AUC-ROC: {roc_auc}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.7615293222688921
AUC-ROC: 0.7608058970958884
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.76      0.86    950161
         1.0       0.09      0.76      0.15     27867

    accuracy                           0.76    978028
   macro avg       0.54      0.76      0.51    978028
weighted avg       0.97      0.76      0.84    978028



In [ ]:
# todo: test the following

# from tpot import TPOTClassifier
# from sklearn.datasets import load_digits
# from sklearn.model_selection import train_test_split

# digits = load_digits()
# X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
#                                                     train_size=0.75, test_size=0.25)

# tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('tpot_digits_pipeline.py')
